In [5]:
import numpy as np
import math
from copy import copy
import sklearn.datasets
from sklearn.svm import SVC

In [6]:
X,y = sklearn.datasets.make_hastie_10_2()
X_train = X[0:8000,:]
y_train = y[0:8000]
X_test = X[8000:,:]
y_test = y[8000:]

# Exercise 1
Implement the AdaBoost ensemble algorithm by completing the following code:

In [4]:
import numpy as np
import math
from sklearn.base import clone

class AdaBoost:
    def __init__(self, weakModel, T):
        #genero T modelli
        self.weakModels = list()        
        self.weakModels.append(weakModel)
        for i in range(0, T):
            self.weakModels.append(copy(weakModel))
        self.T = T
        #array pesi modelli
        self.cw = np.array([])
        return None

    def fit(self, X, y):
        w = np.ones(len(X)) / len(X)

        for t in range(0, self.T):
            self.weakModels[t].fit(X,y,sample_weight=w*len(X)) #alleno il modello t-esimo
            m = self._weakPredict(X,t)

            miss = [int(x) for x in (m != y)]
            miss2 = [x if x==1 else -1 for x in miss]

            err = np.dot(w,miss) / sum(w)

            #err = AdaBoost._computeErr(w,m,y)/sum(w) #calcolo errore pesato
            self.cw = np.append(self.cw,(0.5*math.log((1-err)/err))) #calcolo peso classificatore

            w = self._updateWeights(w,m,y,t) #aggiorno i pesi per gli esempi in base alla classificazione

            #self.weakModels.append(type(self.weakModels[t])(self.param))
            if(t % math.floor(self.T/10) == 0):
                print("Iteration T: {} weighted err:{} Classification Error:{}".format(t, err, AdaBoost.classificationErr(self.predictT(X,t),y)))

    def predictT(self, X, T):
        y = np.array([])
        for x in X:
            s = 0
            for t in range(0, T):
                s += self.weakModels[t].predict(x.reshape(1,-1))*self.cw[t]
            if s > 0:
                y=np.append(y,1)
            else:
                y=np.append(y,-1)
        return y

    def predict(self, X):
        y = np.array([])
        for x in X:
            s = 0
            for t in range(0,self.T):
                s += self.weakModels[t].predict(x.reshape(1,-1))*self.cw[t]
            if s > 0:
                y=np.append(y,1)
            else:
                y=np.append(y,-1)
        return y

    def _weakPredict(self, X, t):
        y = np.array([])
        for x in X:
            y=np.append(y,self.weakModels[t].predict(x.reshape(1,-1)))
        return y

    def _computeErr(w,m,y):
        err = 0
        for i in range(0,len(w)):
            if(m[i]!=y[i]):
                err+=w[i]
        return err

    def _updateWeights(self,w,m,y,t):
        for i in range(0,len(y)):
            w[i]=w[i]*math.exp((-self.cw[t])*y[i]*m[i])
        return w

    def classificationErr(y,Y):
        return 1/2-(np.inner(y,Y)/(2*len(Y)))

In the implementation you are free to assume:
- that the problem is a binary classification problem with labels in $\{-1, +1\}$.
- that the weakModel can fit a weighted sample set by means of the call `weakModel.fit(X,y,sample_weight=w)` where `w` is a vector of length $|y|$.

Test your implementation on the dataset loaded above and using an SVC with a polynomial kernel. 

In [11]:
def classificationErr(y,Y):
    return 1/2-(np.inner(y,Y)/(2*len(Y)))

weakModel = SVC(kernel="poly", degree=3)
adaboost = AdaBoost(weakModel, 100)

adaboost.fit(X_train, y_train)

y_train_ = adaboost.predict(X_train)
y_test_ = adaboost.predict(X_test)

print("Error on training set: ", classificationErr(y_train_, y_train))
print("Error on test set: ", classificationErr(y_test_, y_test))

Iteration T: 0 weighted err:0.34049999999995745 Classification Error:0.49275
Iteration T: 10 weighted err:0.3841514891774316 Classification Error:0.232875
Iteration T: 20 weighted err:0.40228816966093855 Classification Error:0.17912499999999998
Iteration T: 30 weighted err:0.4268418153431585 Classification Error:0.16599999999999998
Iteration T: 40 weighted err:0.4290038536146344 Classification Error:0.153125
Iteration T: 50 weighted err:0.4266811240897086 Classification Error:0.14300000000000002
Iteration T: 60 weighted err:0.44617962543471773 Classification Error:0.1345
Iteration T: 70 weighted err:0.4569049130463657 Classification Error:0.12637500000000002
Iteration T: 80 weighted err:0.45466612909962023 Classification Error:0.12175000000000002
Iteration T: 90 weighted err:0.41474681937805635 Classification Error:0.12087500000000001
Error on training set:  0.11325000000000002
Error on test set:  0.13824999999999998


and evaluate the AdaBoost performances as usual by calculating the classification error. 

**Note 1**:  
since the labels are bound to be in ${+1, -1}$, the classification error can be easily computed as:
$$
   error(y,y') = \frac{1}{2} - \frac{y^T \times y'}{2N},
$$
where $N$ is the total number of examples. The formula can be derived noticing that $y^T \times y'$ calculates the number $N_c$ of examples correctly classified  minus the number $N_{\bar c}$ of examples incorrectly classified. We have then $y^T \times y' = N_c - N_{\bar c}$ and by noticing that $N = N_c + N_{\bar c}$:
$$
   N - y^T \times y' = 2 N_{\bar c} \Rightarrow \frac{N - y^T \times y'}{2 N} = \frac{N_{\bar c}}{N} = error(y,y')
$$

**Note 2**:
do not forget to deepcopy your base model before fitting it to the new data

**Note 3**:
The SVC model allows specifying weights, but it *does not* work well when weights are normalized (it works well when the weights are larger). The following class takes normalized weights and denormalize them before passing them to the SVC classifier:

```python
    class SVC_:
        def __init__(self, kernel="rbf", degree="3"):
            self.svc = SVC(kernel=kernel, degree=degree)

        def fit(self, X,y,sample_weight=None):
            if sample_weight is not None:
                sample_weight = sample_weight * len(X)

            self.svc.fit(X,y,sample_weight=sample_weight)
            return self

        def predict(self, X):
            return self.svc.predict(X)
```

# Exercise 2

Write a weak learner to be used with the AdaBoost algorithm you just wrote. The weak learner that you will implement shall work as follows:

- creates a random linear model $y(x) = \mathbf{w} \cdot \mathbf{x} + t$ by generating the needed weight vector $\mathbf{w}$ and $t$ at random; each weight shall be sampled from U(-1,1);
- it evaluates the weighted loss $\epsilon_t$ on the given dataset and flip the linear model if $\epsilon_t > 0.5$
- at prediction time it predicts +1 if $\mathbf{x} \cdot \mathbf{w} > 0$ it predicts -1 otherwise.

In [2]:
class RandomLinearModel:
    def __init__(self):
        self.w = None
        return None

    def loss(self, y, y_, w):
        miss = [int(x) for x in (y_ != y)]

        err = np.dot(w,miss) / sum(w)

        if(err > 0.5):
            self.w*-1

        return None

    def fit(self,X,y,sample_weight=None):
        self.w = np.random.uniform(-1,1, X[0].size) 
        self.w = np.append(self.w, np.random.randint(-100,100)) # inizializzo i pesi a random qui invece che alla creazione, 
                                                                # dato che Adaboost si limita a copiare il classificatore
        y_=np.zeros(y.size)

        for i in range(0,len(X)):
            y_[i]=self.predict(X[i])

        self.loss(y,y_,sample_weight)

        return None

    def predict(self,X):
        X=np.append(X,1)
        return 1 if np.dot(X,self.w) > 0 else -1

Learn an AdaBoost model using the RandomLinearModel weak learner printing every $K$ iterations the weighted error and the current error of the ensemble (you are free to choose $K$ so to make your output just frequent enough to let you know what is happening but without flooding the console with messages). Evaluate the training and test error of the final ensemble model.

In [8]:
def classificationErr(y,Y):
    return 1/2-(np.inner(y,Y)/(2*len(Y)))

rs = RandomLinearModel()
a = AdaBoost(rs,10000)
a.fit(X_train,y_train)

y_train_ = a.predict(X_train)
y_test_ = a.predict(X_test)

print("Error on training set: ", classificationErr(y_train_, y_train))
print("Error on test set: ", classificationErr(y_test_, y_test))

Iteration T: 0 weighted err:0.48524999999993934 Classification Error:0.48525
Iteration T: 1000 weighted err:0.4959852889194447 Classification Error:0.319
Iteration T: 2000 weighted err:0.49999999999999956 Classification Error:0.248
Iteration T: 3000 weighted err:0.49999999999999994 Classification Error:0.185875
Iteration T: 4000 weighted err:0.4999999999999956 Classification Error:0.15749999999999997
Iteration T: 5000 weighted err:0.5 Classification Error:0.13887500000000003
Iteration T: 6000 weighted err:0.5000000000000018 Classification Error:0.12974999999999998
Iteration T: 7000 weighted err:0.4843367639843449 Classification Error:0.12574999999999997
Iteration T: 8000 weighted err:0.5 Classification Error:0.10925000000000001
Iteration T: 9000 weighted err:0.5212232222526243 Classification Error:0.09687499999999999
Error on training set:  0.08937499999999998
Error on test set:  0.09225


Write few paragraphs about what you think about the experiment and about the results you obtained. 

Trovo che AdaBoost sia un algoritmo di apprendimento assemble molto interessante, rispecchia l'idea dell'unione fa la forza, ogni classificatore aiuta a classificare correttamente una parte degli esempi, che non sono stati classificati correttamente dai suoi predecessori.

Esercizio 1:
In questo esercizio viene usato come classificatore debole, una SVM.
Dai risulati si nota che l'errore di classificazione ensemble diminuisce rapidamente, mentre l'errore pesato del classificatore all'iterazione T sale o scende di poco.
Questo fa capire che un classificatore da solo non sarebbe in grado di classificare in modo adeguato il dataset. Inoltre il fatto che l'errore pesato salga, fa pensare che il classificatore all'iterazione T, si sia specializzato a classificare solamente quegli esempi, che non venivano classificati correttamente dai classificatori generati in precedenza.

Esercizio 2:
In questo esercizio il modello utilizzato è molto semplice, si limita a invertire la pendenza dell'iperpiano ogni volta che classifica in modo sbagliato più di metà del dataset.
La semplicità del modello la si nota anche dal fatto che l'errore pesato all'iterazione T resta sempre intorno a 0.5, quindi significa che riesce giusto a classificare un po meglio di una scelta completamente casuale.
Dato che il modello è molto semplice, le iterazioni richieste per ottenere un risultato soddisfacente sono molte di più rispetto all'esercizio precedente con le SVM, però questo dimostra la potenza di Adaboost, non importa quanto il modello sia semplice se unito a tanti altri, alla fine riuscirà sempre a riprodurre i risultati che si otterrebbero con un modello più complesso.